# IR3 Last column (counts) of the patients for the BL visit.

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import functools

In [3]:
#!pip install dask[complete];
#!pip install --upgrade pandas "dask[complete]"
#python -m pip install "dask[dataframe]" --upgrade

In [12]:
import dask

In [3]:
import dask.dataframe as dd

 We put the main path to the folder where 'counts' files are there.


In [4]:
path1=Path("/scratch/znazari/PPMI_ver_sep2022/RNA_Seq_data/star_ir2/counts/")

 We read the file: in the bl.txt is written the name of all files.

In [5]:
bl_files=pd.read_csv(path1/"bl.txt", header=None )

In [6]:
bl_files.iloc[:,-1:]

,0
0,PPMI-Phase1-IR2.10874.BL.PP0047-6931.5104-SL-0...
1,PPMI-Phase1-IR2.12499.BL.PP0042-3049.5104-SL-1...
2,PPMI-Phase1-IR2.12593.BL.0003149216.5104-SL-13...
3,PPMI-Phase1-IR2.13039.BL.PP0048-8378.5104-SL-1...
4,PPMI-Phase1-IR2.13424.BL.0003180024.5104-SL-18...
...,...
1532,PPMI-Phase2-IR2.65003.BL.0003195560.5104-SL-03...
1533,PPMI-Phase2-IR2.65005.BL.0003451950.5104-SL-26...
1534,PPMI-Phase2-IR2.65008.BL.PP0041-9827.5104-SL-2...
1535,PPMI-Phase2-IR2.70239.BL.0003414527.5104-SL-39...


In [5]:
len(bl_files)

1537

 here we go through all file names and extract their patient IDs and put into a list
 So here we have the list of patient IDs for those whom they have put the data for their base visit

In [8]:
bl_list=[]
for i in range(len(bl_files)):
    bl_list=bl_list+[function_names(bl_files.iloc[i][0])]

 here we read all the files with with base visit from the count folder where we have all the files
 for all the patients and all the visit.
 here we have a list of all files with base visit.

In [9]:
%%time
list_bl_files=[]
for i in range(len(bl_files)):
    list_bl_files=list_bl_files+[dd.read_csv(path1/bl_files.iloc[i][0],skiprows=1,delimiter='\t')]

CPU times: user 15.9 s, sys: 10.8 s, total: 26.7 s
Wall time: 2min 16s
